In [1]:
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
from keras import layers
from keras import models

In [2]:
model = models.Sequential()

model.add(layers.Conv1D(64, kernel_size=20, activation='relu',input_shape=(150, 1)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, kernel_size=20, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.LSTM(512))
model.add(layers.Dense(120))
model.add(layers.Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])          
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 131, 64)           1344      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 65, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 64)            81984     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 23, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1181696   
_________________________________________________________________
dense (Dense)                (None, 120)               61560     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [3]:
import os
import cv2

In [4]:
train_dir = 'train'
test_dir = 'test'
vali_dir = 'validation'
train_typea_dir = os.path.join(train_dir,'type_a')
train_typeb_dir = os.path.join(train_dir,'type_b')
train_typec_dir = os.path.join(train_dir,'type_c')
train_typed_dir = os.path.join(train_dir,'type_d')
train_typee_dir = os.path.join(train_dir,'type_e')

In [ ]:
'''
train = cv2.imread("train/type_a/s1.png")
trainX = train[0:320, 0:119]
array_of_train = []
cut_img = []
#print(train[239].size)
for cut_range in range(120, 145):
    cut_img.append((train[cut_range]))
    np.array(cut_img) 
    #print(cut_range)
    print(np.array(cut_img).size)

#cut_img=np.array(cut_img)
#print(np.array(cut_img).size)
#print(cut_img)
#array_of_train.append(cut_img)
cv2.imshow('My Image', train)
cv2.waitKey(0)
cv2.destroyAllWindows()
''''''

In [5]:
array_of_train = []
array_of_label = []
# this function is for read image,the input is directory name
def read_directory(directory_name):
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in range (int(len(os.listdir(r"./"+directory_name))/2)):
        #print(filename) #just for test
        #img is used to store the image data
        train = cv2.imread(directory_name + "/s" + str(filename+1)+"_init.png", cv2.IMREAD_GRAYSCALE)
        label = cv2.imread(directory_name + "/s" + str(filename+1)+".png", cv2.IMREAD_GRAYSCALE)
        train = train[0:320, 0:120]
        label = label[0:320, 0:120]

        array_of_train.append(train)
        array_of_label.append(label)
   
        #print(label)
    print(np.array(array_of_train[0]).shape)
    #cv2.imshow('My Image', array_of_train[20])
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return array_of_train, array_of_label


In [6]:
train_data = read_directory("train/type_a")

(240, 120)


In [7]:
np.array(array_of_train).shape

(72, 240, 120)

In [8]:
from sklearn.preprocessing import MinMaxScaler
data_reshape = np.array(array_of_train).reshape(-1,1).astype('float32')
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(data_reshape)
np.shape(data_reshape)

(2073600, 1)

In [9]:
look_back = 150
test_size = 1000000
train, test = training_set_scaled[:-test_size], training_set_scaled[-test_size-look_back:] 

In [10]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [11]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], 150, 1))
testX = np.reshape(testX, (testX.shape[0], 150, 1))
#print(trainX.shape)
#print(testX.shape)
#, trainY
#, testY

In [ ]:
hist = model.fit(trainX, trainY, epochs=100, batch_size=1000, validation_data=(testX, testY))

Epoch 1/100
 121/1074 [==>...........................] - ETA: 12:44 - loss: 0.6188 - acc: 0.9430

In [ ]:
import matplotlib.pyplot as plt

acc = hist.history['acc']
val_acc = hist.history['val_acc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
#plt.savefig('accu.png')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
#plt.savefig('loss.png')

plt.show()
